In [1]:
import pandas as pd
pd.set_option('display.width', 120)
pd.set_option('display.max_rows', 5000)
import warnings
warnings.filterwarnings('ignore')

path1 = "https://raw.githubusercontent.com/Soyoung-Yoon/bigdata/main/"
path2 = "https://raw.githubusercontent.com/Soyoung-Yoon/data_01/main/"

#### 7-1) 지역별 공간점수

다음 절차에 따라 문제를 해결하시오.

* bed와 bath의 값이 모두 0이 아닌 행만 선택한다.
* 각 행(row)별로 bed * 1.5 + bath * 2 의 값을 '공간점수'라고 정의한다.
* 지역(zip_code)별로 공간점수의 평균을 계산한 후, 지역별 평균 공간점수가 가장 높은 7개 지역의 가격(price) 평균을 반올림하여 정수로 구하라.

In [ ]:
df = pd.read_csv(path1 + "housing01.csv")
# print(df.head(), df.shape, sep="\n") # (104667, 12)
# print(df.info())

# bed와 bath의 값이 모두 0이 아닌 행만 선택한다.
df = df.loc[(df['bed'] > 0) & (df['bath'] > 0)].copy()
# print(df.shape) # 0은 없음

# 각 행(row)별로 bed * 1.5 + bath * 2 의 값을 '공간점수'라고 정의한다.
df['공간점수'] = (df['bed'] * 1.5) + (df['bed'] * 2)
# print(df.head())

# 지역(zip_code)별로 공간점수의 평균을 계산한 후, 지역별 평균 공간점수가 가장 높은 7개 지역의 가격(price) 평균을 반올림하여 정수로 구하라.
df2 = df.groupby('zip_code')[['공간점수']].mean().reset_index().sort_values('공간점수', ascending=False)[:7]
result = round(df2.merge(df[['zip_code', 'price']], how="left", on='zip_code')['price'].mean())
# print(result) # 6434486

6434486


#### 7-2) 이벤트 발생 빈도

다음 절차에 따라 문제를 해결하시오.

* (년, 월)별로 발생한 이벤트의 개수를 확인하여, 이벤트 발생 개수 TOP3의 이벤트 수 합을 구하여 정수로 출력하시오.
* TOP3는 가장 큰 값 3개를 의미한다.
* start_datetime : 발생 날짜/시간
* end_datetime : 종료 날짜/시간

In [ ]:
df = pd.read_csv(path2 + "event_log_03.csv")
# print(df.head(), df.shape, sep="\n") # (1000, 4)
# print(df.isna().sum().to_frame().T) # 결측치 없음

# (년, 월)별로 발생한 이벤트의 개수를 확인하여, 이벤트 발생 개수 TOP3의 이벤트 수 합을 구하여 정수로 출력하시오.
df['start_datetime'] = pd.to_datetime(df['start_datetime'])
df['end_datetime'] = pd.to_datetime(df['end_datetime'])
df['year'] = df['start_datetime'].dt.year
df['month'] = df['start_datetime'].dt.month
# print(df.head())
df['counts'] = df['event']
df2 = df.groupby(['year','month','event'])[['counts']].count().reset_index().sort_values('counts', ascending=False).head(3)
result = int(df2['counts'].sum())
# print(result) # 32


32


#### 7-3) 이벤트 발생 시간분석

다음 절차에 따라 문제를 해결하시오.

* 이벤트가 시작된 시각(start_datetime)을 기준으로 이벤트가 가장 많이 발생한 시(hour)를 구하시오.
* 해당 시각에 발생한 모든 이벤트의 value 값을 합산하고, 그 합계를 반올림하여 정수로 출력하시오.

In [ ]:
df = pd.read_csv(path2 + "event_log_03.csv")
# print(df.head(), df.info(), sep="\n")
# print(df.isna().sum().to_frame().T) # 결측치 없음

# 이벤트가 시작된 시각(start_datetime)을 기준으로 이벤트가 가장 많이 발생한 시(hour)를 구하시오.
df['start_datetime'] = pd.to_datetime(df['start_datetime'])
df['hour'] = df['start_datetime'].dt.hour
# print(df.head())
result = df.groupby('hour')['event'].count().reset_index().sort_values('event', ascending=False)['hour'].values[0]
# print(result) # 2

2


#### 7-4) 월별 이벤트 발생 빈도

다음 절차에 따라 문제를 해결하시오.

* (년, 월)별로 발생한 이벤트의 개수를 확인하여, 년별로 가장 많은 이벤트가 발생한 월에 해당하는 데이터 개수의 합을 구하여 정수로 출력하시오.

In [ ]:
df = pd.read_csv(path2 + "event_log_03.csv")
# print(df.head(), df.info(), sep="\n")
# print(df.isna().sum().to_frame().T) # 결측치 없음

#  (년, 월)별로 발생한 이벤트의 개수를 확인하여
df['start_datetime'] = pd.to_datetime(df['start_datetime'])
df['year'] = df['start_datetime'].dt.year
df['month'] = df['start_datetime'].dt.month

# 년별로 가장 많은 이벤트가 발생한 월에 해당하는 데이터 개수의 합을 구하여 정수로 출력
df['counts'] = df['event']
result = int(df.pivot_table(index='year', columns='month',  values='counts', aggfunc="count").max().sum())
# print(result) # 378


378


#### 7-5a) 지속시간의 분위수 분석 다음 절차에 따라 문제를 해결하시오.

* 지속시간 기준 75분위수(=제3사분위수) 이상에 해당하는 데이터의 지속시간(duration) 합계를 초단위 정수로 출력한다.
* 지속시간(duration) = end_datetime - start_datetime

In [ ]:
df = pd.read_csv(path2 + "event_log_03.csv")
# print(df.head(), df.info(), sep="\n")

# 지속시간 기준 75분위수(=제3사분위수) 이상에 해당하는 데이터의 지속시간(duration) 합계를 
# 초단위 정수로 출력한다.
df['start_datetime'] = pd.to_datetime(df['start_datetime'])
df['end_datetime'] = pd.to_datetime(df['end_datetime'])
df['duration'] = df['end_datetime'] - df['start_datetime']

result = int(df[df['duration'] >= df['duration'].quantile(0.75)]['duration'].sum().total_seconds())
print(result) # 133254

133254


#### 7-5b) 지속시간의 정확한 개수 사용

다음 절차에 따라 문제를 해결하시오.

* 지속시간 기준 정확히 상위 25%에 해당하는 데이터 개수만 뽑아 지속시간(duration) 합계를 초단위 정수로 출력한다.
* 지속시간(duration) = end_datetime - start_datetime

In [ ]:
df = pd.read_csv(path2 + "event_log_03.csv")
# print(df.head(), df.info(), sep="\n")

# 지속시간 기준 정확히 상위 25%에 해당하는 데이터 개수만 뽑아 지속시간(duration) 합계를 초단위 정수로 출력한다.
df['start_datetime'] = pd.to_datetime(df['start_datetime'])
df['end_datetime'] = pd.to_datetime(df['end_datetime'])
df['duration'] = df['end_datetime'] - df['start_datetime']

dur25 = len(df[df['duration']>=df['duration'].quantile(0.75)])
result = int(df['duration'].nlargest(dur25, keep='first').sum().total_seconds())
# print(result) # 133254

133254
